In [ ]:
#Bash script to generate the BAM list for all BAM files
import os

# Directory where the BAM files are located
bam_directory = 'path/to/BAM/files/data'
# Directory where the output files should be saved
output_directory = 'path/to/save/list'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Get a list of all BAM files in the directory
bam_files = sorted([f for f in os.listdir(bam_directory) if f.endswith('.bam')])

# Create an individual file for each BAM file
for idx, bam_file in enumerate(bam_files, start=1):
    file_path = os.path.join(bam_directory, bam_file)
    output_file = os.path.join(output_directory, f'ID{idx}_list.lst')
    
    with open(output_file, 'w') as f:
        f.write(f"ID{idx},{file_path}\n")

print(f"Formatted list files saved to {output_directory}")

In [ ]:
#Bash script to generate the region list for all BAM files

# Generate lists of chrX where X ranges from 1 to 22
chromosomes = ['chr' + str(x) for x in range(1, 23)]

# Iterate through each chromosome and create a corresponding file
for chromosome in chromosomes:
    filename = chromosome + '.lst'
    with open(filename, 'w') as f:
        f.write(chromosome + '\n')
    print(f"Created file: {filename}")

In [ ]:
#Cluster job script for Data Preprocessing

#!/bin/bash

# Define paths
path="/path/to/cloned/dir/of/Monopogen"
list_dir="path/to/list"
base_output_dir="path/to/save/output"
scripts_dir="path/to/preprocess/scripts"

# Set the library path
export LD_LIBRARY_PATH="${path}/apps:$LD_LIBRARY_PATH"
echo $LD_LIBRARY_PATH

# Ensure base output directory exists
mkdir -p $base_output_dir

# Loop through each list file, sorted numerically, and submit a SLURM job for each
for list_file in $(ls ${list_dir}/ID*_list.lst | sort -V); do
    # Extract the ID from the list file name
    filename=$(basename -- "$list_file")
    id="${filename%_list.lst}"

    # Create a unique output directory for this dataset
    output_dir="${base_output_dir}/${id}"
    mkdir -p $output_dir

    echo "Preprocessing ${list_file} with output to ${output_dir}..."

    # Run the preprocessing job script
    sbatch ${scripts_dir}/Preprocess.sh ${path}/src/Monopogen.py ${list_file} ${output_dir} 8

done

echo "Preprocessing submitted for all files."


In [ ]:
#Cluster job script for Germline Call
# Only 5 datasets were used due to technical issues

#!/bin/bash

# Number of datasets
num_datasets=5

# Template file
template_file="path/to/alt_germline.sh"

# Loop over each dataset
for dataset_id in $(seq 1 $num_datasets); do
  # Create a SLURM script for the current dataset
  job_script="run_germline_c20_ID${dataset_id}.sh"
  sed "s/{{DATASET_ID}}/${dataset_id}/g" $template_file > $job_script

  # Submit the job script
  echo "Submitting job script for Dataset ID: $dataset_id"
  sbatch $job_script
done


In [ ]:
#For Somatic Calling, the code remains the same as Monopogen Git Repository has provided
#This was done for all 5 IDs individually
#Scripts and Outputs are saved under respective folder in respective ID folders

In [ ]:
#Result Analysis

import pandas as pd
import gzip

# Load the filtered_ID1.csv file to get the list of IDs
filtered_ids_df = pd.read_csv('path/to/filtered_ID1.csv')

# Display the dataframe
filtered_ids_df.head(), filtered_ids_df.shape

In [ ]:
# Load the chr20.gl.filter.hc.cell.mat.gz file
with gzip.open('path/to/chr20.gl.filter.hc.cell.mat.gz', 'rt') as f:
    chr20_df = pd.read_csv(f, sep='\t', header=None)

# Rename the first four columns
chr20_df.rename(columns={0: "chr", 1: "pos", 2: "ref_ale", 3: "alt_ale"}, inplace=True)

# Display the first few rows of the dataframe
chr20_df.head()

In [ ]:
# Extract the 'chr' and 'pos' columns from the filtered_ids_df
filtered_chr = filtered_ids_df['chr'].iloc[0]
filtered_pos = filtered_ids_df['pos'].iloc[0]

# Filter the chr20_df based on the 'chr' and 'pos' columns
filtered_chr20_df = chr20_df[(chr20_df['chr'] == filtered_chr) & (chr20_df['pos'] == filtered_pos)]

# Display the filtered dataframe
filtered_chr20_df

In [ ]:
# Define the order of columns: 1-4, 19-20, and then the rest
columns_to_display = list(filtered_chr20_df.columns[:4])+list(filtered_chr20_df.columns[18:])

# Reorder the dataframe based on the new column order
reordered_df = filtered_chr20_df[columns_to_display]

# Display the reordered dataframe
display(reordered_df)

In [ ]:
#Read the file to replace the column header with single-cell sequences
cell_seq_file = pd.read_csv("path/to/chr20.cell_snv.cellID.filter.csv", usecols=['cell'])

#Transpose the rows to columns
cell_seq = cell_seq_file.T

#Display the single-cell sequences
display(cell_seq)

# Extract the headers from the transposed cell sequence dataframe
new_headers = cell_seq.iloc[0, :].tolist()

# Create new column names for reordered_df starting from the 6th column onward
new_column_names = list(reordered_df.columns[:4]) + new_headers

# Check if the lengths match
if len(new_column_names) == len(reordered_df.columns):
    # Rename the columns in reordered_df
    reordered_df.columns = new_column_names
    # Display the reordered dataframe with the new column names
    display(reordered_df.head())

In [ ]:
#Categorize Mutation Types

# Function to classify mutation status
def mutation(value):
    if value == '0/0':
        return '0'
    elif value in ['1/0', '0/1', '1/1']:
        return '1'
    else:
        return value 

# Apply the function to all columns in the reordered dataframe
mutation_df = reordered_df.map(mutation)

# Display the dataframe
print(mutation_df)

In [ ]:
# Number of Cells with this particular Mutation

# Create a new column indicating the number of mutations in each row
mutation_df['mutation_count'] = mutation_df.apply(lambda row: sum(1 for val in row if val == '1'), axis=1)

final_df = mutation_df

# Display the final dataframe
print(final_df)

# Create a column with information of col 1-4 is merged to one

def generate_names(df):
    df['snvID'] = df['snvID'] = df.apply(lambda row: f"{row[0]}:{row[1]}:{row[2]}:{row[3]}", axis=1)
    return df

snv_id_set = generate_names(final_df)

print(snv_id_set)

In [ ]:
#No of cells having particular mutations

# Identify columns with 'mutation' values
mutation_columns = mutation_df.columns[mutation_df.eq('1').any()]

# Convert the index object to a list
mutation_columns_list = mutation_columns.tolist()

# Create a new dataframe from the list
cell_mut_df = pd.DataFrame(mutation_columns_list, columns=['Mutation_Columns'])

# Display the new dataframe
print(cell_mut_df)

In [ ]:
#Extract the sequence and get info from Randolph metadata

# Load Randolph metadata TSV file
import re
randolph_metadata = pd.read_csv('path/to/Randolph_singlecell_metadata.tsv', sep='\t')

# Extract sequence from cell_mut_df
cell_mut_df['Extracted_CellID'] = cell_mut_df['Mutation_Columns'].apply(lambda x: re.match(r'([^-\d]+)', x).group(1))

# Extract sequence from randolph_metadata
randolph_metadata['Extracted_CellID'] = randolph_metadata['CellID'].apply(lambda x: re.search(r'_([^_]+)$', x).group(1))

# Merge the DataFrames on the extracted cell IDs
merged_df = pd.merge(cell_mut_df, randolph_metadata, on='Extracted_CellID', how='left')

display(merged_df[['Extracted_CellID', 'celltype', 'Batch']])

In [ ]:
# Filter the results to the batch ID for the partiular dataset

# [batch ID data retrieved from sra metadata file] for eg: B1_c1

# Define the batch identifier to filter by
batch_to_filter = 'B1_c1'

# Filter the DataFrame for the specific batch identifier
filtered_df = merged_df[merged_df['Batch'] == batch_to_filter]
count_rows = len(filtered_df)

# Display the filtered DataFrame
print(f'Total number of cells with this mutation = {count_rows}')
display(filtered_df)

In [ ]:
#Cell Type Count for Mutated 

# Create DataFrame
df = pd.DataFrame(filtered_df)

# Group by SOC_indiv_ID and then count occurrences of each cell type within each group
cell_type_counts = df.groupby('SOC_indiv_ID')['celltype'].value_counts()

# List unique cell types
unique_cell_types = df['celltype'].unique()

# Display results
print("Cell Type Counts by SOC_indiv_ID:")
print(cell_type_counts)

In [ ]:
# Retrieve age from donor metadata

# Read the donor metadata file
donor_metadata_df = pd.read_csv('path/to/donor_metadata_with_batchid.csv')

#Selecting the required columns
donor_age_df = donor_metadata_df[['indiv_ID', 'batchID', 'age']]
# Merge the DataFrames
result_df = pd.merge(filtered_df, donor_age_df, left_on=['SOC_indiv_ID', 'Batch'], right_on=['indiv_ID', 'batchID'], how='left')

# Display the result
count_rows = len(result_df)
print(f'No. of Rows = {count_rows}')
display(result_df[['Extracted_CellID', 'SOC_indiv_ID','celltype', 'Batch', 'age', 'SOC_infection_status']])

In [ ]:
#For check total cells present in the sample

# Extract column headers
headers = mutation_df.columns.tolist()

# Slice headers after the 4th column
headers_after_4th = headers[4:5947]

total_cells_list = [{'Column': header} for header in headers_after_4th]
cell_total_df = pd.DataFrame(total_cells_list)

# Display the new DataFrame
count_rows = len(cell_total_df)
print(f'Total number of columns after the 4th column = {count_rows}')

# Load Randolph metadata TSV file
randolph_metadata = pd.read_csv('path/to/Randolph_singlecell_metadata.tsv', sep='\t')

# Extract sequence from cell_mut_df
cell_total_df['Extracted_CellID'] = cell_total_df['Column'].apply(lambda x: re.match(r'([^-\d]+)', x).group(1))

# Extract sequence from randolph_metadata
randolph_metadata['Extracted_CellID'] = randolph_metadata['CellID'].apply(lambda x: re.search(r'_([^_]+)$', x).group(1))

# Merge the DataFrames on the extracted cell IDs
merged_total_df = pd.merge(cell_total_df, randolph_metadata, on='Extracted_CellID', how='left')

count_rows = len(merged_total_df)
print(f'Total number of cells = {count_rows}')
print(merged_total_df)

In [ ]:
#For calculating total cells present in this sample

# Define the batch identifier to filter by
filter_batch = 'B1_c1'

# Filter the DataFrame for the specific batch identifier
sort_cell_total_df = merged_total_df[merged_total_df['Batch'] == filter_batch]
count_rows = len(sort_cell_total_df)

# Display the filtered DataFrame
print(f'Total number of cells = {count_rows}')
display(sort_cell_total_df[['Extracted_CellID','celltype', 'Batch', 'SOC_indiv_ID', 'SOC_infection_status']])

In [ ]:
# Total Cell Type Count

# Create DataFrame
total_df = pd.DataFrame(sort_cell_total_df)

# Group by SOC_indiv_ID and then count occurrences of each cell type within each group
total_cell_type_counts = total_df.groupby('SOC_indiv_ID')['celltype'].value_counts()

# Display results
print("Cell Type Counts:", total_cell_type_counts)

In [ ]:
#Mutation Rate Calculation

# Create DataFrames
mutation_cell_type_df = pd.DataFrame(cell_type_counts)
total_cell_type_df = pd.DataFrame(total_cell_type_counts)

# Group by SOC_indiv_ID and celltype to sum total occurrences
total_cell_type_counts = total_cell_type_df.groupby(['SOC_indiv_ID', 'celltype']).sum().reset_index().rename(columns={'count': 'total_cell_type_count'})

# Group by SOC_indiv_ID and celltype to sum mutated occurrences
mutation_cell_type_counts = mutation_cell_type_df.groupby(['SOC_indiv_ID', 'celltype']).sum().reset_index().rename(columns={'count': 'mut_cell_type_count'})

# Merge the DataFrames on 'SOC_indiv_ID' and 'celltype'
mutation_rate_df = pd.merge(total_cell_type_counts, mutation_cell_type_counts, on=['SOC_indiv_ID', 'celltype'], how='left')

# Define the human chr20 length
chromosome_length = 64444167 

# Calculate the mutation rate for each cell type
mutation_rate_df['mutation_rate'] = mutation_rate_df['mut_cell_type_count'] / mutation_rate_df['total_cell_type_count'] / chromosome_length

# Display results
count_rows = len(mutation_rate_df)
print(f'Total number of rows = {count_rows}')
display("Cell Type Counts by SOC_indiv_ID and Mutation Rates:", mutation_rate_df)

In [ ]:
# Filter and Remove the NaN values from Mutation Rate Calculation

# Sort the DataFrame by SOC_indiv_ID and celltype
mutation_rate_df_sorted = mutation_rate_df.sort_values(by=['SOC_indiv_ID', 'celltype'])

# Remove rows with NaN values
mutation_rate_df_clean = mutation_rate_df_sorted.dropna(subset=['mut_cell_type_count'])

# Display results and the new DataFrame
count_rows = len(mutation_rate_df_clean)
print(f'Total number of rows = {count_rows}')
display("Cell Type Counts by SOC_indiv_ID and Mutation Rates (Sorted):", mutation_rate_df_clean)

In [ ]:
#Retrieve Age information

# Merge the DataFrames on SOC_indiv_ID and indiv_ID
age_cell_df = pd.merge(mutation_rate_df_clean, donor_age_df, left_on='SOC_indiv_ID', right_on='indiv_ID', how='left')

# Drop unnecessary columns from donor_age_df if needed (e.g., 'indiv_ID' column)
age_cell_df = age_cell_df.drop(columns=['indiv_ID'])

# Display the updated DataFrame with age information
count_rows = len(age_cell_df)
print(f'Total number of rows = {count_rows}')

# Filter rows where batchID is 'B1_c1'
result_filtered_df1 = age_cell_df.loc[age_cell_df['batchID'] == 'B1_c1']

# Display the filtered DataFrame and count of rows
count_filtered_rows = len(result_filtered_df1)
print(f'Total number of rows for batchID B1_c1 = {count_filtered_rows}')
display("Filtered DataFrame with Age Information for batchID B1_c1:", result_filtered_df1)

# Save the dataframe to .csv file
result_filtered_df1.to_csv('path/to/save/file/result_filtered_df1.csv')

In [ ]:
#Plot Graphs
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Ensure the DataFrame has no NaN values in 'age' or 'mutation_rate'
result_filtered_df = result_filtered_df1.dropna(subset=['age', 'mutation_rate'])

# Get unique cell types
cell_types = result_filtered_df['celltype'].unique()

# Define a color palette for cell types
palette = sns.color_palette("husl", len(cell_types))  # Using a distinct color palette

# Create a figure and axis
plt.figure(figsize=(12, 8))

# Plot each cell type with different colors
for i, cell_type in enumerate(cell_types):
    cell_subset = result_filtered_df[result_filtered_df['celltype'] == cell_type]
    
    # Check if cell_subset is not empty
    if cell_subset.empty:
        continue
    
    # Plot scatter plot
    sns.scatterplot(x='age', y='mutation_rate', data=cell_subset, color=palette[i], marker='o', label=cell_type)
    
    # Check if there is variation in 'age' values
    if len(cell_subset['age'].unique()) > 1:
        # Fit line of best fit
        slope, intercept, r_value, p_value, std_err = stats.linregress(cell_subset['age'], cell_subset['mutation_rate'])
        
        # Generate line of best fit
        age_range = np.linspace(cell_subset['age'].min(), cell_subset['age'].max(), 100)
        mutation_rate_fit = intercept + slope * age_range
        
        # Plot line of best fit with label
        plt.plot(age_range, mutation_rate_fit, color=palette[i], linestyle='--',
                 label=f'{cell_type} Fit: y={slope:.2f}x+{intercept:.2f}, R²={r_value**2:.2f}')
    else:
        print(f"Not enough variation in 'age' values for cell type: {cell_type}")

# Set titles and labels
plt.title('Mutation Rate vs Age')
plt.xlabel('Age')
plt.ylabel('Mutation Rate')
plt.legend()

# Adjust layout and show plot
plt.tight_layout()
plt.show()

In [ ]:
#Create a summary table of the graph

# Ensure the DataFrame has no NaN values in 'age' or 'mutation_rate'
result_filtered_df = result_filtered_df1.dropna(subset=['age', 'mutation_rate'])

# Get unique cell types
cell_types = result_filtered_df['celltype'].unique()

# Prepare lists to hold the summary information
summary_data = {
    'Cell Type': [],
    'Slope': [],
    'Intercept': [],
    'R-squared': [],
    'P-value': [],
    'Std Error': []
}

# Calculate regression parameters for each cell type
for cell_type in cell_types:
    cell_subset = result_filtered_df[result_filtered_df['celltype'] == cell_type]
    
    # Check if there is variation in 'age' values
    if len(cell_subset['age'].unique()) > 1:
        # Perform linear regression
        slope, intercept, r_value, p_value, std_err = stats.linregress(cell_subset['age'], cell_subset['mutation_rate'])
        
        # Append the results to the summary data
        summary_data['Cell Type'].append(cell_type)
        summary_data['Slope'].append(slope)
        summary_data['Intercept'].append(intercept)
        summary_data['R-squared'].append(r_value**2)
        summary_data['P-value'].append(p_value)
        summary_data['Std Error'].append(std_err)
    else:
        print(f"Not enough variation in 'age' values for cell type: {cell_type}")
        
# Create a DataFrame from the summary data
summary_df1 = pd.DataFrame(summary_data)

# Display the summary table
summary_df1[['Cell Type', 'Slope', 'Intercept', 'R-squared']]

In [ ]:
# Combine all datasets to plot graph for combined mutation rate vs age
import pandas as pd

# List of file paths
file_paths = ["define/path/to/result_filtered_df1.csv", "../result_filtered_df2.csv" ..... "./nth file.csv"]

# Load all CSV files into DataFrames
dataframes = [pd.read_csv(file_path) for file_path in file_paths]

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Drop rows with NaN values in 'age' or 'mutation_rate'
combined_df = combined_df.dropna(subset=['age', 'mutation_rate'])

# Ensure 'celltype' column is treated as categorical
combined_df['celltype'] = combined_df['celltype'].astype('category')

display(combined_df)


In [ ]:
# Remove the cell types with < 3 data points

# Count data points for each cell type
cell_type_counts = combined_df['celltype'].value_counts()

# Filter cell types with at least 3 data points
valid_cell_types = cell_type_counts[cell_type_counts >= 3].index

# Filter combined DataFrame to include only valid cell types
filtered_df = combined_df[combined_df['celltype'].isin(valid_cell_types)]

display(filtered_df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

# Define a color palette for valid cell types
cell_types = filtered_df['celltype'].unique()
palette = sns.color_palette("husl", len(cell_types))

# Initialize a list to hold summary data
summary_data = []

# Create the plot
plt.figure(figsize=(12, 8))

# Plot each valid cell type with different colors
for i, cell_type in enumerate(cell_types):
    cell_subset = filtered_df[filtered_df['celltype'] == cell_type]
    
    # Check if cell_subset is not empty
    if cell_subset.empty:
        continue
    
    # Check if there is variation in 'age' values
    if len(cell_subset['age'].unique()) > 1:
        # Fit line of best fit
        slope, intercept, r_value, _, _ = stats.linregress(cell_subset['age'], cell_subset['mutation_rate'])
        
        # Append to summary data
        summary_data.append({
            'Cell Type': cell_type,
            'Slope': slope,
            'Intercept': intercept,
            'R²': r_value**2
        })
        
        # Generate line of best fit
        age_range = np.linspace(cell_subset['age'].min(), cell_subset['age'].max(), 100)
        mutation_rate_fit = intercept + slope * age_range
        
        # Plot line of best fit with label
        plt.plot(age_range, mutation_rate_fit, color=palette[i], linestyle='--',
                 label=f'{cell_type} Fit: y={slope:.2f}x+{intercept:.2f}, R²={r_value**2:.2f}')
    else:
        print(f"Not enough variation in 'age' values for cell type: {cell_type}")

# Plot scatter points for each cell type
sns.scatterplot(data=filtered_df, x='age', y='mutation_rate', hue='celltype', palette=palette, marker='o')

# Set titles and labels
plt.title('Combined Mutation Rate vs Age Across Cell Types')
plt.xlabel('Age')
plt.ylabel('Mutation Rate')
plt.legend(title='Cell Type')

# Adjust layout to prevent overlap and show plot
plt.tight_layout()
plt.show()

# Create a DataFrame from the summary data
summary_df = pd.DataFrame(summary_data)